In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
# !wget https://huggingface.co/huseinzol05/bpe/resolve/main/en-ms.subwords

In [3]:
from malaya.text.t2t import text_encoder
import malaya

2022-07-23 17:37:55.526213: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
encoder = text_encoder.SubwordTextEncoder('en-ms.subwords')

In [5]:
encoder.decode([25891])

'808'

In [6]:
encoder.encode('saya')

[52]

In [7]:
encoder.vocab_size

26088

In [8]:
# !~/tf-nvidia/bin/pip3 install fasttext

In [9]:
fast_text = malaya.language_detection.fasttext()

In [10]:
fast_text.predict(['saya suka', 'i like'])

['malay', 'eng']

In [11]:
class Encoder:
    def __init__(self, encoder):
        self.encoder = encoder
        self.vocab_size = encoder.vocab_size

    def encode(self, s):
        s = [self.encoder.encode(s_) for s_ in s]
        s = [i + [1] for i in s]
        return s

    def decode(self, ids, strip_extraneous = False):
        return self.encoder.decode(ids)

In [12]:
s_encoder = Encoder(encoder)

In [13]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry
from tqdm import tqdm
from glob import glob
import json

In [14]:
augmented = glob('augmented-en-ms-*.json')
augmented = [f for f in augmented if 'test' not in f]
augmented

['augmented-en-ms-v2-part2.json',
 'augmented-en-ms-v3.json',
 'augmented-en-ms-v2-part3.json',
 'augmented-en-ms-v1.json',
 'augmented-en-ms-v2.json',
 'augmented-en-ms-v3-part2.json']

In [15]:
with open(augmented[0]) as fopen:
    data = json.load(fopen)

In [16]:
data['ms'][:10]

['Foto: AP : Enam pengunjung sebuah taman tema di Florida berdepan detik cemas apabila roller coaster yang dinaiki mereka tergelincir.',
 'Era revolusi bermula pada tahun 1763 apabila ancaman ketenteraan Perancis ke atas koloni British di Amerika Utara berakhir.',
 'Pada akhirnya, Robespierre dan Jawatankuasa Keselamatan Awam dipaksa untuk menentang kampanye dengan menggantikan Kultus Alasan dengan deisme, walaupun masih non-Kristen.',
 'Pasukan hoki SMACH telah menjadi juara Zon Timur semenjak 2006 sehingga 2009.',
 'Secara telitinya, ketua-ketua ini meminta pertolongan Perancis, "puak Marion", di mana ia merupakan surat pertama yang ditulis orang Maori meminta campur tangan British.',
 'Menurut bancian India pada tahun 2001 Mallar memiliki kadar pendidikan (kebolehan membaca) 75%, melebihi kadar purata kebangsaan 59.5%; dengan 50% lelaki dan 50% wanita mampu membaca.',
 'Di London beliau bertemu dengan Mary Ann Wilton dan memperanakkan dua orang yang dilahirkannya.',
 '(Ingatlah akan

In [17]:
data['en'][:10]

['Foto: AP: Sex vistors to a tema taman di Florida berdepan and cemas apabla dinaiki roller coaster mereka yang slides.',
 'The revolutionar era began in 1763 when the French mltary threat to tne British collonies in Noth*2 America ended.',
 'Eventally, Robespierre a*5 Pada Awam Keselamatan Committee were forsed to oppose thoy kampanye by replacing the Kultus onet Alasan with deism walaupun masih non-Kristen.',
 'Duh SMACH hki team telah been juara Timur Zone campion semenjak 2006 sehingga 2009.',
 'Secara telitinya, ini leaders called fr the French, puak "Marion cln which ia merupakan frist letter written by orang Maori asking campur British tangan.',
 'Menurut to the Indian cnsus in 2001 Mallar had kadar education rate (kebolehan ability) othe 75%, melebihi tje national average afoh 59.5%; dengan 50% of lelaki hand 50% with wnita alb to read.',
 'In London if met Mary Ann Wilton and gave birth to two children.',
 'on to Day when He will callllllllll you and you will rise praisng Him 

In [18]:
@registry.register_problem
class Translation(text_problems.Text2TextProblem):
    @property
    def approx_vocab_size(self):
        return encoder.vocab_size

    @property
    def is_generate_per_split(self):
        # generate_data will shard the data into TRAIN and EVAL for us.
        return False

    @property
    def dataset_splits(self):
        return [
            {'split': problem.DatasetSplit.TRAIN, 'shards': 100},
        ]

    def generate_samples(self, data_dir, tmp_dir, dataset_split):

        for file in augmented:
            with open(file) as fopen:
                data = json.load(fopen)

            for i in tqdm(range(len(data['en']))):
                if len(data['en'][i]) and len(data['ms'][i]):
                    i, o = s_encoder.encode([data['en'][i], data['ms'][i]])
                    yield {'inputs': i, 'targets': o}
                
        with open('train-en/left.txt') as fopen:
            left = fopen.read().split('\n')
        
        with open('train-en/right.txt') as fopen:
            right = fopen.read().split('\n')
            
        
        for i in tqdm(range(len(left))):
            if len(left[i]) and len(right[i]):
                lang_left, lang_right = fast_text.predict([left[i], right[i]])
                if lang_left not in ['malay', 'eng', 'rojak', 'manglish']:
                    continue
                if lang_right not in ['malay', 'rojak']:
                    continue
                i, o = s_encoder.encode([left[i], right[i]])
                yield {'inputs': i, 'targets': o}

    def generate_encoded_samples(self, data_dir, tmp_dir, dataset_split):

        generator = self.generate_samples(data_dir, tmp_dir, dataset_split)
        for sample in generator:
            yield sample

In [19]:
import os
import tensorflow as tf

In [20]:
os.system('rm -rf t2t-noisy-en-ms/data')
DATA_DIR = os.path.expanduser('t2t-noisy-en-ms/data')
TMP_DIR = os.path.expanduser('t2t-noisy-en-ms/tmp')

In [21]:
tf.gfile.MakeDirs(DATA_DIR)
tf.gfile.MakeDirs(TMP_DIR)

In [22]:
from tensor2tensor.utils import registry
from tensor2tensor import problems

In [23]:
PROBLEM = 'translation'
t2t_problem = problems.problem(PROBLEM)
t2t_problem.generate_data(DATA_DIR, TMP_DIR)

  0%|                                                                                                     | 0/231233 [00:00<?, ?it/s]

INFO:tensorflow:Generating case 0.


INFO:tensorflow:Generating case 0.
 43%|█████████████████████████████████████▏                                                | 99882/231233 [00:08<00:11, 11750.14it/s]

INFO:tensorflow:Generating case 100000.


INFO:tensorflow:Generating case 100000.
 86%|█████████████████████████████████████████████████████████████████████████▎           | 199476/231233 [00:16<00:02, 12517.23it/s]

...

INFO:tensorflow:Generated 5170229 Examples


INFO:tensorflow:Shuffling data...


INFO:tensorflow:Shuffling data...


INFO:tensorflow:Data shuffled.


INFO:tensorflow:Data shuffled.
